# Description:
>这里是基于机器学习和深度学习的训练的基础上进行的又一次尝试，机器学习是用的多项式贝叶斯，深度学习用的bert，但是效果看起来还是有点不太理想，多项式贝叶斯的训练正确度35%左右，而Bert也只能到达70%左右，我初步怀疑是bert采用的数据集的数量不够，所以我下一步还想进一步增大数据集，用bert再运行。 这里尝试用另一种模型，word2vec与LSTM进行结合的多分类模型。 [参考博客](https://github.com/DLLXW/MultiClassify_LSTM_ForChinese)

In [2]:
"""导入包"""
# 基础包
import numpy as np
import pandas as pd

# 处理语言句子的包
import re
import jieba
from gensim.models.word2vec import Word2Vec

# 数据集处理
import keras.utils
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
import imp
import yaml
import sys

# 创建LSTM模型
from keras import utils as np_utils
from keras.models import Sequential
from keras.models import model_from_yaml
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Dropout, Activation
from keras.models import load_model
# 多进程处理
import multiprocessing


import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


# 准备数据集
 根据博客中的描述，这里的数据集需要在原来的基础上进行修改，老师给的只是一个.xlsx文件，但是在这里需要进行拆分，修改
> * 首先，读入训练集，把多标签的样本拆成几个，都变成单标签
> * 根据标签的类型进行分组，分完组之后把样本写入.txt文件，由于是六分类问题，文件名以六个类标签为名

In [2]:
"""读入原始数据集"""
df = pd.read_excel('../Dataset/train.xlsx', index_col=False, names=['data','label'],encoding='utf-8')
df['data'] = df['data'].astype(str)

In [3]:
for d, l in df.iterrows():
    l.label = l.label.replace('[','').replace(']','').replace("'",'').split(',')

In [4]:
df1 = pd.DataFrame()
df2 = pd.DataFrame()

In [5]:
for i in range(df.shape[0]):
    temp = df.iloc[i]
    if (len(temp['label'])>1):
        for item in temp['label']:
            df2 = df2.append({"data":temp['data'], "label":item.strip()}, ignore_index=True)
    else:
        df1 = df1.append(temp, ignore_index=True)

In [6]:
print(df1.shape)
print(df2.shape)

(5364, 2)
(1310, 2)


In [7]:
for i,v in df1['label'].items():
    if (type(v) == list):
        df1['label'][i] = v[0]

In [8]:
df = pd.concat([df1, df2], axis=0, ignore_index=True)

In [9]:
print("dfshape:", df.shape)

dfshape: (6674, 2)


In [10]:
df.head()

,data,label
0,【 恶魔们 的 高跟鞋 】 gasoline glamour shoes 集锦 ~ ~ ！ ！ ！,None
1,经过 长途跋涉 ， 终于 到 了 威尼斯 。 晩上 从 机场 搭乘 boat taxl ， ...,Happiness
2,hello kitty 」 停 不 了 的 爱 。,Happiness
3,唱 错词 了 哈哈 好 可爱 坑 死 日本人 - 评论 视频 ： 1991 年 【 beyo...,Happiness
4,我 是 能量 满 分 选手 ， 我们 又 聊到 这么 晚 才 回家 ， 但是 如果 还 有 ...,Happiness


In [11]:
"""统计标签个数，并准备写入.txt"""
df['label'].value_counts()

None         1896
Happiness    1823
Sadness      1086
Surprise      651
Fear          648
Anger         570
Name: label, dtype: int64

In [12]:
none = pd.DataFrame()
Happiness = pd.DataFrame()
Sadness = pd.DataFrame()
Surprise = pd.DataFrame()
Fear = pd.DataFrame()
Anger = pd.DataFrame()

In [13]:
df['label'].values

array(['None', 'Happiness', 'Happiness', ..., 'Sadness', 'Happiness',
       'Fear'], dtype=object)

In [14]:
"""遍历df，开始筛选，划分"""
for i in range(df.shape[0]):
    temp = df.iloc[i]
    if temp['label'] == 'None':
        none = none.append(temp, ignore_index=True)
    elif temp['label'] == 'Happiness':
        Happiness = Happiness.append(temp, ignore_index=True)
    elif temp['label'] == 'Sadness':
        Sadness = Sadness.append(temp, ignore_index=True)
    elif temp['label'] == 'Surprise':
        Surprise = Surprise.append(temp, ignore_index=True)
    elif temp['label'] == 'Fear':
        Fear = Fear.append(temp, ignore_index=True)
    else:
        Anger = Anger.append(temp, ignore_index=True)

In [15]:
"""写入.txt文件"""
none['data'].to_csv('newdata/None.txt', index=False)
Happiness['data'].to_csv('newdata/Happiness.txt', index=False)
Sadness['data'].to_csv('newdata/Sadness.txt', index=False)
Surprise['data'].to_csv('newdata/Surprise.txt', index=False)
Fear['data'].to_csv('newdata/Fear.txt', index=False)
Anger['data'].to_csv('newdata/Anger.txt', index=False)

# 导入数据集并进行清洗
> 上一步已经把需要处理的数据处理好了，下面导入上面的数据集，并且进行清理，去掉特殊符号（标点符号，数字，空格等）只保留汉字
><br>
> 这里的数据集在去掉特殊符号的基础上，利用jieba分词进行了处理，这样就做好了word2vec的输入，方便下面基于word2vec进行词向量的构建。

In [16]:
"""清理数据，把无关的字符全部去掉"""
def clean_data(rpath,wpath):
    # coding=utf-8
    pchinese = re.compile('([\u4e00-\u9fa5]+)+?')
    f = open(rpath,encoding="UTF-8")
    fw = open(wpath, "w",encoding="UTF-8")
    for line in f.readlines():
        m = pchinese.findall(str(line))
        if m:
            str1 = ''.join(m)
            str2 = str(str1)
            fw.write(str2)
            fw.write("\n")
    f.close()
    fw.close()

In [17]:
"""导入数据集"""
def loadfile():
    none = []
    happiness = []
    sadness = []
    surprise = []
    fear = []
    anger = []
 
    with open('newdata_clean/None.txt',encoding='UTF-8') as f:
        for line in f.readlines():
            none.append(list(jieba.cut(line, cut_all=False, HMM=True))[:-1])
    with open('newdata_clean/Happiness.txt',encoding='UTF-8') as f:
        for line in f.readlines():
            happiness.append(list(jieba.cut(line, cut_all=False, HMM=True))[:-1])
        f.close()
    with open('newdata_clean/Sadness.txt',encoding='UTF-8') as f:
        for line in f.readlines():
            sadness.append(list(jieba.cut(line, cut_all=False, HMM=True))[:-1])
        f.close()
    with open('newdata_clean/Surprise.txt',encoding='UTF-8') as f:
        for line in f.readlines():
            surprise.append(list(jieba.cut(line, cut_all=False, HMM=True))[:-1])
        f.close()
    with open('newdata_clean/Fear.txt',encoding='UTF-8') as f:
        for line in f.readlines():
            fear.append(list(jieba.cut(line, cut_all=False, HMM=True))[:-1])
        f.close()
    with open('newdata_clean/Anger.txt',encoding='UTF-8') as f:
        for line in f.readlines():
            anger.append(list(jieba.cut(line, cut_all=False, HMM=True))[:-1])
        f.close()
        
        X_Vec = np.concatenate((none, happiness, sadness,surprise, fear, anger))

    y = np.concatenate((np.zeros(len(none), dtype=int),
                        np.ones(len(happiness), dtype=int),
                        2*np.ones(len(sadness), dtype=int),
                        3*np.ones(len(surprise), dtype=int),
                        4*np.ones(len(fear),  dtype=int),
                        5*np.ones(len(anger), dtype=int)))

    return X_Vec, y

In [18]:
print("开始清洗数据................")
clean_data('newdata/None.txt','newdata_clean/None.txt')
clean_data('newdata/Happiness.txt','newdata_clean/Happiness.txt')
clean_data('newdata/Sadness.txt','newdata_clean/Sadness.txt')
clean_data('newdata/Surprise.txt','newdata_clean/Surprise.txt')
clean_data('newdata/Fear.txt','newdata_clean/Fear.txt')
clean_data('newdata/Anger.txt','newdata_clean/Anger.txt')
print("清洗数据完成................")

开始清洗数据................
清洗数据完成................


In [19]:
print("开始导入数据................")
X_Vec, y=loadfile()
print("导入数据完成................")

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ZHONGQ~1\AppData\Local\Temp\jieba.cache


开始导入数据................


Loading model cost 0.584 seconds.
Prefix dict has been built successfully.


导入数据完成................


In [20]:
print(X_Vec.shape)
print(y.shape)

(6664,)
(6664,)


# 基于word2vec构建词向量
> 计算机只能处理数字，为了将词语送给计算机处理，自然而然的要想办法将词语进行编码(对图像而言，编码无非就是一个一/三维的像素矩阵)，但是对自然语言的编码就困难了，想要把意思相近的词语编码成相似的向量可不是一件容易的事情，于是有人想到word2vec,其实它就干了一件事儿，把词语嵌入(编码)到一个高维空间(向量)，这些向量之间隐含了词语之间的关系。这个向量怎么来呢？答：训练得来。
>
而之前的机器学习只是单独的统计了词频作为了特征，并没有关心词与词之间的关系，而这里的word2vec向量，是把词语之间的关系包含在了一个高维空间中。
这里利用python 的gensim库

In [21]:
voc_dim = 150 #word的向量维度
min_out = 4  #单词出现频率数
window_size = 7 #
cpu_count = multiprocessing.cpu_count()

def word2vec_train(X_Vec):
    model_word = Word2Vec(size=voc_dim,
                     min_count=min_out,
                     window=window_size,
                     workers=cpu_count,
                     iter=100)
    model_word.build_vocab(X_Vec)
    model_word.train(X_Vec, total_examples=model_word.corpus_count, epochs=model_word.iter)
    model_word.save('model/Word2Vec_java.pkl')

    print(len(model_word.wv.vocab.keys()))
    input_dim = len(model_word.wv.vocab.keys()) + 1 #频数小于阈值的词语统统放一起，编码为0
    embedding_weights = np.zeros((input_dim, voc_dim))
    w2dic={}
    for i in range(len(model_word.wv.vocab.keys())):
        embedding_weights[i+1, :] = model_word[list(model_word.wv.vocab.keys())[i]]
        w2dic[list(model_word.wv.vocab.keys())[i]]=i+1
    return input_dim,embedding_weights,w2dic

In [22]:
print("开始构建词向量................")
input_dim,embedding_weights,w2dic = word2vec_train(X_Vec)
print("构建词向量完成................")

开始构建词向量................
5863
构建词向量完成................


# 划分数据集

In [23]:
def data2inx(w2indx,X_Vec):
    data = []
    for sentence in X_Vec:
        new_txt = []
        for word in sentence:
            try:
                new_txt.append(w2indx[word])
            except:
                new_txt.append(0)

        data.append(new_txt)
    return data

In [24]:
index = data2inx(w2dic,X_Vec)
index2 = sequence.pad_sequences(index, maxlen=voc_dim )
x_train, x_test, y_train, y_test = train_test_split(index2, y, test_size=0.2)
y_train = keras.utils.to_categorical(y_train, num_classes=6)
y_test = keras.utils.to_categorical(y_test, num_classes=6)

# 创建模型并进行训练
>训练好了词向量，下一步就是把词向量合成句子的向量，这一会利用RNN来把高维的句向量编重新编码，注二分类用的sigmoid，现在多分类需要改为softmax

In [25]:
imp.reload(sys)
np.random.seed()

In [26]:
"""构建LSTM模型"""
lstm_input = 150#lstm输入维度
voc_dim = 150 #word的向量维度
def lstm(input_dim, embedding_weights):
    model = Sequential()
    model.add(Embedding(output_dim=voc_dim,
                        input_dim=input_dim,
                        mask_zero=True,
                        weights=[embedding_weights],
                        input_length=lstm_input))
    model.add(LSTM(256, activation='softsign'))
    model.add(Dropout(0.5))
    model.add(Dense(6))
    model.add(Activation('softmax'))
    return model

In [27]:
"""训练LSTM模型"""

voc_dim = 150 #word的向量维度
lstm_input = 150#lstm输入维度
epoch_time = 10#epoch
batch_size = 32 #batch

def train_lstm(model, x_train, y_train, x_test, y_test):
    print('Compiling the Model...')
    model.compile(loss='binary_crossentropy',#hinge
                  optimizer='adam', metrics=['mae', 'acc'])

    print("Train..." )
    model.fit(x_train, y_train, batch_size=batch_size, epochs=epoch_time, verbose=1)

    print("Evaluate...")
    print(model.predict(x_test))
    score = model.evaluate(x_test, y_test,
                           batch_size=batch_size)

    yaml_string = model.to_yaml()
    with open('model/lstm_java.yml', 'w') as outfile:
        outfile.write(yaml.dump(yaml_string, default_flow_style=True))
    model.save('model/lstm_java_total.h5')
    print('Test score:', score)

In [28]:
model = lstm(input_dim, embedding_weights)
train_lstm(model, x_train, y_train, x_test, y_test)

Compiling the Model...
Train...
Epoch 1/10
5331/5331 [==============================] - 54s 10ms/step - loss: 0.4070 - mean_absolute_error: 0.2508 - acc: 0.8368
Epoch 2/10
5331/5331 [==============================] - 53s 10ms/step - loss: 0.3661 - mean_absolute_error: 0.2285 - acc: 0.8478
Epoch 3/10
5331/5331 [==============================] - 54s 10ms/step - loss: 0.3311 - mean_absolute_error: 0.2075 - acc: 0.8610
Epoch 4/10
5331/5331 [==============================] - 53s 10ms/step - loss: 0.2907 - mean_absolute_error: 0.1834 - acc: 0.8804
Epoch 5/10
5331/5331 [==============================] - 56s 11ms/step - loss: 0.2486 - mean_absolute_error: 0.1578 - acc: 0.8980
Epoch 6/10
5331/5331 [==============================] - 56s 10ms/step - loss: 0.2051 - mean_absolute_error: 0.1309 - acc: 0.9175
Epoch 7/10
5331/5331 [==============================] - 53s 10ms/step - loss: 0.1702 - mean_absolute_error: 0.1088 - acc: 0.9312
Epoch 8/10
5331/5331 [==============================] - 58s 11ms/

# 测试
> 上面的模型训练好了之后，下面是用自己的实例进行推理。输入句子--->保留汉字--->分词--->word2Vec编码--->送入模型推理

In [1]:
"""导入模型"""
voc_dim = 150
######
model_word=Word2Vec.load('model/Word2Vec_java.pkl')

NameError: name 'Word2Vec' is not defined

In [30]:
input_dim = len(model_word.wv.vocab.keys()) + 1
embedding_weights = np.zeros((input_dim, voc_dim))
w2dic={}
for i in range(len(model_word.wv.vocab.keys())):
    embedding_weights[i+1, :] = model_word [list(model_word.wv.vocab.keys())[i]]
    w2dic[list(model_word.wv.vocab.keys())[i]]=i+1

model = load_model('model/lstm_java_total.h5')

pchinese = re.compile('([\u4e00-\u9fa5]+)+?')

label={0:"None", 1:"Happiness", 2:"Sadness", 3:"Surprise", 4:"Fear", 5:"Anger"}
#in_stc=["明天","就要","考试","我","特别","紧张","一点","都","没有","复习"]
in_str = "真是气死我了"
in_stc=''.join(pchinese.findall(in_str))

in_stc=list(jieba.cut(in_stc,cut_all=True, HMM=False))

new_txt=[]

data=[]
for word in in_stc:
    try:
        new_txt.append(w2dic[word])
    except:
        new_txt.append(0)
data.append(new_txt)

data=sequence.pad_sequences(data, maxlen=voc_dim )
pre=model.predict(data)[0].tolist()
print(pre)
print("输入：")
print("  ",in_str)
print("        ")
print("输出:")
print("  ",label[pre.index(max(pre))])


[0.04378584027290344, 0.06542085111141205, 0.07086724787950516, 0.3303574323654175, 0.08224285393953323, 0.40732577443122864]
输入：
   真是气死我了
        
输出:
   Anger


# 产生提交的结果
> 这一块就是根据老师的测试集进行测试了， 测试集也需要先把多余的符号给去掉，然后借鉴上面的这个测试把测试集的标签打上，然后写入到文件

In [31]:
"""读入测试集"""
test = pd.read_excel('../DataSet/test.xlsx')
test.to_csv('newdata/test.txt', index=False)

In [32]:
test.head()

,我 收藏 了 ： why me-李宇 春
0,嗯 、 够钟 翻归 了 。 明天 顶住 个 翻工 。 —— at last 《 你 瞒 我 ...
1,《 鹿鼎记 ： 龙脉 传说 》 : 12月 23日 觉醒 公测 ， 追寻 龙脉 遗迹 ， 梦...
2,我 正在 收听 sarah connor 的 单曲 《 just one last danc...
3,呢 杯野 ！ 好＂勒 ＂ 啊 ！ blue mountain ！
4,马天宇 是 不 是 个 gay ？ 难道 没有 解答 ？


In [33]:
"""处理测试集，把多余的符号给去掉，生成新的测试集,然后读入"""

clean_data('newdata/test.txt','newdata_clean/test.txt')
newtest = pd.read_csv("newdata_clean/test.txt", header=None)

In [34]:
print(newtest.shape)

(1199, 1)


In [11]:
newtest.head()

,0
0,我收藏了李宇春
1,嗯够钟翻归了明天顶住个翻工你瞒我瞒无言的亲亲亲侵袭我心仍宁愿亲口讲你累得很如除我以外在你心还...
2,鹿鼎记龙脉传说月日觉醒公测追寻龙脉遗迹梦回鹿鼎江湖金顶门的很强力啊不过还是我技高一筹啊哈哈
3,我正在收听的单曲来自豆瓣私人兆赫
4,呢杯野好勒啊


In [14]:
newtest.values

array([['我收藏了李宇春'],
       ['嗯够钟翻归了明天顶住个翻工你瞒我瞒无言的亲亲亲侵袭我心仍宁愿亲口讲你累得很如除我以外在你心还多出一个人你瞒住我我亦瞒住我太合衬无人车我翻归我行出大堂门口挥一挥衣袖打车是一件他妈奢侈的玩意'],
       ['鹿鼎记龙脉传说月日觉醒公测追寻龙脉遗迹梦回鹿鼎江湖金顶门的很强力啊不过还是我技高一筹啊哈哈'],
       ...,
       ['发表了博文可爱死了拉'],
       ['过去只有自己照顾自己的时候掌声很少的时候没有人给他那么多建议的时候大多数时间只能自己沉默的时候自己一个人背着骑着机车在夜幕中穿越台北街头的时候可能无形中早就练就了他一个人做决定的习惯他成熟的很早很快很坦然很坚决他一直是这样我也将一直支持萧敬腾摘自贴吧'],
       ['哎呦你能笑得再贱一点儿么']], dtype=object)

In [37]:
"""把测试的写成一个函数"""
def compute_sample_label(word):

     # 建立映射字典
    label_mapping = {0:"None", 1:"Happiness", 2:"Sadness", 3:"Surprise", 4:"Fear", 5:"Anger"}
    
    input_dim = len(model_word.wv.vocab.keys()) + 1
    embedding_weights = np.zeros((input_dim, voc_dim))
    w2dic={}
    for i in range(len(model_word.wv.vocab.keys())):
        embedding_weights[i+1, :] = model_word [list(model_word.wv.vocab.keys())[i]]
        w2dic[list(model_word.wv.vocab.keys())[i]]=i+1

    model = load_model('model/lstm_java_total.h5')

    pchinese = re.compile('([\u4e00-\u9fa5]+)+?')

    label={0:"None", 1:"Happiness", 2:"Sadness", 3:"Surprise", 4:"Fear", 5:"Anger"}
    #in_stc=["明天","就要","考试","我","特别","紧张","一点","都","没有","复习"]
    in_str = word
    in_stc=''.join(pchinese.findall(in_str))

    in_stc=list(jieba.cut(in_stc,cut_all=True, HMM=False))

    new_txt=[]

    data=[]
    for word in in_stc:
        try:
            new_txt.append(w2dic[word])
        except:
            new_txt.append(0)
    data.append(new_txt)

    data=sequence.pad_sequences(data, maxlen=voc_dim )
    pre=model.predict(data)[0].tolist()
    
    sample_label = []
    for i in range(len(pre)):
        if (pre[i]>0.3):
            sample_label.append(label_mapping[i])
    
    return sample_label

In [39]:
"""产生结果"""
labels = []

for word in newtest.values:
    samplelabel = compute_sample_label(word[0])
    labels.append(samplelabel)

KeyboardInterrupt: 

# 总结与参考
> 参考的一个博客:[LSTM对中文文本的多分类](https://zhuanlan.zhihu.com/p/97745819)  在这个博客上的基础上修改而来的，自己唯一做的工作就是把老师的数据集，通过处理，换成这个模型能够处理的数据集，然后去进行分类。<br>
> 如果想要用测试集进行测试，还是需要把测试集进行数据预处理一下，然后再进行测试。这一个的效果或许会好一点，甚至比bert模型的那个效果还好，可能是因为词比单个字的特征把控起来容易。<br>
>
记录一下知识点：
> * 关于jieba
> * 关于gensim
> * Word2Vec
> * 代码的具体细节